In [ ]:
#4号任务nlp day2

#1、下载数据集
#中文数据集：THUCNews
#THUCNews数据子集

#完
#下载地址
#THUCNews数据子集：https://pan.baidu.com/s/1hugrfRu 密码：qfud
#英文数据集：IMDB数据集 http://ai.stanford.edu/~amaas/data/sentiment/


In [2]:
#2、下载IMDB数据集（影评文本）与探索
#tf自带IMDB数据集，需用keras库辅助下载
#下载地址与方式参考TensorFlow官方教程：
#https://tensorflow.google.cn/tutorials/keras/basic_text_classification
#https://www.kesci.com/home/project/5b6c05409889570010ccce90
    
import tensorflow as tf
from tensorflow import keras
import numpy as np
print(tf.__version__)


1.8.0


In [3]:
#下载IMDB
imdb = keras.datasets.imdb
(train_data,train_labels),(test_data,test_lables) = imdb.load_data(num_words = 10000)
#num_words = 10000 意为保留训练数据中频次在10000位的字词，而少见词舍弃

17465344/17464789 [==============================] - 8s 0us/step


In [6]:
#探索分析数据
#打印看下数据量有多大
print("训练集数据数量：{},训练集标签数量：{}".format(len(train_data),len(train_labels)))


训练集数据数量：25000,训练集标签数量：25000


In [10]:
#打印第一条看下
#print(train_data[0])
#输出[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 6……
#可知影评数据果然文字已处理为数字表示


In [12]:
len(train_data[0]),len(train_data[1]),len(train_data[3])
#注意：第一条和第二条第三条字数都是不等的

(218, 189, 550)

In [13]:
#还可用辅助函数将数字表示的词转回文字
#参考https://tensorflow.google.cn/tutorials/keras/basic_text_classification#download_the_imdb_dataset
#其它文字若要进行神经网络训练要用One-hot编码转成数字表示，像最初下载的样本一样。

In [26]:
#处理样本不均衡问题（即，每条长度不等问题）
#可以填充数组，使它们都具有相同的长度
#或使用预处理函数 pad_sequences将长度标准化

train_data = keras.preprocessing.sequence.pad_sequences(train_data,
            #value=word_index["<PAD>"],
            padding='post',
            maxlen=256)
test_data = keras.preprocessing.sequence.pad_sequences(
    test_data,
    #value=word_index['PAD'],
    padding = 'post',
    maxlen=256
)
len(train_data[0]),len(train_data[1])
#(256, 256)，经过预处理，样本长度一样了。

(256, 256)

In [28]:
#print(train_data[0])
#打印第1条发现末尾已补了0…… 0    0    0    0]

In [ ]:
#3、THUCNews数据集下载和探索
#已在1中下载，
#下面做下数据探索分析
#THUCNews是由清华NLP组提供的新闻文本分类数据集
#参考博客中的数据集部分和预处理部分：https://blog.csdn.net/u011439796/article/details/77692621
#参考代码：https://github.com/gaussic/text-classification-cnn-rnn/blob/master/data/cnews_loader.py

#数据集分析介绍：

#数据集使用了其中的10个分类，每个分类6500条，总共65000条新闻数据。
#类别：体育, 财经, 房产, 家居, 教育, 科技, 时尚, 时政, 游戏, 娱乐
'''
数据集划分如下：

训练集: 5000*10
验证集: 500*10
测试集: 1000*10
'''

#预处理
#data/cnews_loader.py为数据的预处理文件。
#代码见https://github.com/gaussic/text-classification-cnn-rnn/blob/master/data/cnews_loader.py
'''
read_file():读文件
build_vocab():建词汇表，避免每次重复处理
read_vocab():读上一步存储的词汇表转为{词:id}
read_category():将分类目录固定，转为{分类:id}
to_words():将由id表示的数据转回文字
preocess_file():将数据集从文字转换为固定长度的id序列表示
batch_iter():为神经网络训练准备经过shuffle洗牌的批次的数据
'''
#预处理后数据格式如下：
'''
x_train	[50000, 600]	y_train	[50000, 10]
x_val	[5000, 600]	y_val	[5000, 10]
x_test	[10000, 600]	y_test	[10000, 10]
'''

In [ ]:
#4. 学习召回率、准确率、ROC曲线、AUC、PR曲线这些基本概念
#参考1：https://www.imooc.com/article/48072
#参考：https://blog.csdn.net/zk_j1994/article/details/78478502

召回率Recall：召回率是从关注领域中，召回目标类别的比例
Recall = TP/TP+FN  即 正例真/正例真+负例假
可用混淆矩阵热力图直观查看。

正确率：正确率也叫预测精度，是评估捕获的成果中目标成果所占得比例
精度 = TP+FN / TP + FN + TN + FP (即所有可能结果？模糊认识，待确认)
【遗留问题：参考：准确率(Accuracy), 精确率(Precision), 召回率(Recall)和 F1-Measure，待改
https://www.cnblogs.com/sddai/p/5696870.html】

有时召回率和正确率是此消彼长，相互矛盾的，可以通过ROC曲线来找二者的平衡

ROC曲线

ROC曲线常用于二分类问题中的模型比较，
是用来平衡准确率和召回率的曲线，
主要表现为一种真正例率 (TPR) 和假正例率 (FPR) 的权衡。
ROC曲线可以看作是随着阈值的不断移动，所有样本中正例与负例之间的“对抗”。曲线越靠近左上角，意味着越多的正例优先于负例，模型的整体表现也就越好。

AUC （ROC  Area）：ROC曲线与x和y轴围成的面积。AUC可以综合衡量一个预测模型的好坏，即综合了精确度precision和召回率recall两个指标。

#但AUC计算很麻烦，有人用简单的F-score来代替。F-score计算方法很简单：
F-score=(2*precision*recall)/(precision+recall)

ROC的有点：是可视化好，一目了解，可以用混淆矩阵热力图画出来看下。
缺点：只关心正例的情况不适合。

#所以出现了
PR曲线（Precision Recall）：即预测精度准确度曲线，和ROC一样，也用了TPR Recall召回率 来计算评估模型，也可以用AUC面积来衡量分类器效果，不同是，PR的指票都聚集在正例，精度曲线，ROC是召回率曲线。因为好多类别不平衡的情况多关心正例的情况多，所以PR比ROC被认为优一些。





